In [16]:
!pip install -qU pinecone-client openai langchain gradio

In [17]:
import openai
import warnings
import pinecone
import warnings

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Pinecone
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.agents import Tool
from langchain.agents import initialize_agent

In [ ]:
openai.api_key = ""
PINECONE_API_KEY = ""  # find next to api key in console
PINECONE_ENV = ""  # find next to api key in console

llm = ChatOpenAI(openai_api_key=openai.api_key, temperature=0.0)

EMBEDDING_MODEL_NAME = 'text-embedding-ada-002'
warnings.filterwarnings('ignore')

embed = OpenAIEmbeddings(
    model=EMBEDDING_MODEL_NAME,
    openai_api_key=openai.api_key
)

index_name = 'semantic-search-openai'
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='dotproduct',
        dimension=1536  # 1536 dim of text-embedding-ada-002
    )
# connect to index
index = pinecone.Index(index_name)
print('Pinecone index status is', index.describe_index_stats())

text_field = "text"
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=1,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool when answering queries about '
            'the SEC files'
        )
    )
]

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [20]:
print(agent("Whats the total revenue of Tesla for 2022?"))



> Entering new AgentExecutor chain...
{
    "action": "Knowledge Base",
    "action_input": "What is the total revenue of Tesla for 2022?"
}
Observation: The total revenue of Tesla for 2022 was $81.46 billion, representing an increase of $27.64 billion compared to the prior year.
Thought:{
    "action": "Final Answer",
    "action_input": "The total revenue of Tesla for 2022 was $81.46 billion, representing an increase of $27.64 billion compared to the prior year."
}

> Finished chain.
{'input': 'Whats the total revenue of Tesla for 2022?', 'chat_history': [], 'output': 'The total revenue of Tesla for 2022 was $81.46 billion, representing an increase of $27.64 billion compared to the prior year.'}


In [21]:
import gradio as gr

def query_vector_db(query):
    return agent(query)

sec_file_query_demo_app = gr.Interface(
    fn=query_vector_db,
    inputs=gr.Textbox(lines=1, placeholder="Please Enter the SEC file search query here !"),
    outputs="text",
)
sec_file_query_demo_app.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>